# RMI Detail Transactions Project

## 1.  Load Transactional Data, SalesForce Data, Mavenlink Data, load packages, and convert to Dataframes

I am using three reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

Search in Netsuite search bar using "sea" for Mavenlink Time Audit

In Salesforce, use Ginny's link, but filter for "ALL" in Opportunity Record Type.  Also include Revenue Date:  greater or equal to LAST FISCAL YEAR.

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


In [2]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults240.xls', excelHandler)
df1 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])



In [3]:


excelHandler = ExcelHandler()
parse('../Data/raw/MavenlinkTimeAuditResults101.xls', excelHandler)
dfMavenlink = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [4]:
excelHandler = ExcelHandler()
parse('../Data/raw/Employees588.xls', excelHandler)
employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [5]:
#EmployeeNameandID = employee_list[['Name', 'External ID', 'Job Title']]

In [6]:
#EmployeeNameandID['External ID'].replace('', np.nan, inplace=True)

In [7]:
#len(EmployeeNameandID)

In [8]:
#EmployeeNameandID.to_excel('originallist.xlsx')

In [9]:
#employee_id_list = employee_list['External ID'].tolist()

In [10]:
#employee_id_list = list(filter(None, employee_id_list))

In [11]:
#print(len(employee_id_list))

In [12]:
df1['Memo'] = df1['Memo'].astype('string')

In [13]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [14]:
#all_ids = '|'.join(employee_id_list)

In [15]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

In [16]:
#df1['External ID'] = df1['Memo'].str.findall(all_ids).apply(return_first)

In [17]:
#employee_name_list = employee_list['Name'].tolist()

In [18]:
#employee_name_list = list(filter(None, employee_name_list))

In [19]:
#all_names = '|'.join(employee_name_list)

In [20]:
#all_names = all_names + "|Agnis Li|Anna Ledden|Greg Hales|Rachel Hohensee|Li Ma"

In [21]:
#df1['Match ID Names'] = df1['Memo'].str.findall(all_names).apply(return_first)

In [22]:
#df1 = pd.merge(df1, EmployeeNameandID, on = 'External ID', how='left')

In [23]:
#df1['Match ID Names'].replace('', np.nan, inplace=True)

In [24]:
#df1['Name'].replace('', np.nan, inplace=True)

In [25]:
#df1['Name'].fillna(df1['Match ID Names'], inplace=True)

In [26]:
#df1['Name'].replace('Agnis Li', 'Agnes Li', inplace=True)
#df1['Name'].replace('Anna Ledden', 'Anna  Ledden', inplace=True)
#df1['Name'].replace('Rachel Hohensee', 'Rachel T Hohensee', inplace=True)

In [27]:
#len(df1)

## Mavenlink Data Manipulation

In [28]:
dfMavenlink['Month'] = pd.DatetimeIndex(dfMavenlink['Date']).month
dfMavenlink['Month'] = dfMavenlink['Month'].map({7: 'Jul 2020', 8: "Aug 2020", 9: "Sep 2020", 10: "Oct 2020", 11: "Nov 2020", 12: "Dec 2020", 1: "Jan 2021", 2: "Feb 2021", 3: "Mar 2021", 4: "Apr 2021", 5: "May 2021", 6: "Jun 2021"})

In [29]:
dfMavenlink['Minutes'] =  dfMavenlink['Duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) * 1)
dfMavenlink['Hours'] =  dfMavenlink['Minutes'] / 60

In [30]:
dfMavenlink['Grant'].replace('', np.nan, inplace=True)
dfMavenlink['Initiative'].replace('', np.nan, inplace=True)
dfMavenlink['Employee'].replace('', np.nan, inplace=True)
dfMavenlink['Case/Task/Event'].replace('', np.nan, inplace=True)
dfMavenlink['Billable'].replace('', np.nan, inplace=True)

In [31]:
dfMavenlink['Grant'].fillna("No Grant", inplace=True)
dfMavenlink['Initiative'].fillna("No Initiative", inplace=True)
dfMavenlink['Grant'].fillna("Employee", inplace=True)
dfMavenlink['Case/Task/Event'].fillna("Case/Task/Event", inplace=True)
dfMavenlink['Billable'].fillna("Billable", inplace=True)

In [32]:
dfMavenlink['Employee Hours'] = dfMavenlink.groupby(['Employee','Month'])['Hours'].transform('sum')



In [33]:
dfMavenlink['Percent of Total Monthly Hours'] = dfMavenlink['Hours'] / dfMavenlink['Employee Hours']

In [34]:
dfMavenlink.head()

,Internal ID,External ID,Employee,Duration,Program,Initiative,Client,Grant,Restriction,Case/Task/Event,...,Billable,Date,Item,Type,Actual Time Entry Cost,Month,Minutes,Hours,Employee Hours,Percent of Total Monthly Hours
0,9607850,1535266785,Radhika Lalit,8:00,Global Climate Finance,GCF - Center for a Climate-Aligned Finance,*Restricted : Center for Climate Aligned Finance,Wells Fargo - GCF CCAF FY20,1-Unrestricted,Admin (Project Task),...,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,631.861,Jul 2020,480,8.0,360.0,0.022222
1,9609942,1536835065,Emily Goldfield,4:00,India,IN - India,Children's Investment Fund Foundation : CIFF T...,No Grant,1-Unrestricted,Project management (Project Task),...,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,264.776,Jul 2020,240,4.0,184.0,0.021739
2,9627632,1531726455,Holly Leonard,4:00,Urban Transformation,UT - Admin,*Unrestricted : Urban Transformation [Holiday]...,Bloomberg America's Pledge FY20,1-Unrestricted,PTO-AP2020 (Project Task),...,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,230.71,Jul 2020,240,4.0,202.5,0.019753
3,9593084,1532866255,James Newcomb,2:00,Strategic Analysis and Engagement,SAE - Admin,*Unrestricted : Strategic Analysis and Engag...,Wells Fargo - EES Phase 2 FY20,1-Unrestricted,Meetings (Project Task),...,No,2020-07-01T00:00:00,Professional Personnel,Actual Time,453.404,Jul 2020,120,2.0,184.0,0.010870
4,9606292,1535531785,Shelley Backstrom,2:00,Islands,IS - Islands,*Restricted : Bermuda,Anonymous - Bermuda FY20,1-Unrestricted,Electricity (Project Task),...,Yes,2020-07-01T00:00:00,Professional Personnel,Actual Time,104.538,Jul 2020,120,2.0,184.0,0.010870


In [35]:
#dfMavenlink.to_excel('Output_Mavenlink.xlsx')

## Extract Time Allocation From Memo

In [36]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [37]:
all_allocations = '|'.join(allocations)

In [38]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [39]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [40]:
lookups = pd.read_excel('../Data/raw/lookups.xlsx')

In [41]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [42]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [43]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [44]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [45]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [46]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
ALLOCATION                          object
Project Allocations                 object
Rollup Account                      object
Account Dig

## Add Reclass Field and Closeout field

In [47]:
account_exceptions = ['To reclass', 'Reclass']

In [48]:
Accounting_Moves = '|'.join(account_exceptions)

In [49]:
closeout = 'close out'

In [50]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [51]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [52]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [53]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

## ADDING TITLES INSTEAD OF NAMES

In [54]:
#EmployeeNameandID['Job Title'].value_counts()

In [55]:
#uniquetitles = EmployeeNameandID['Job Title'].unique()

In [56]:
#Titleslist = uniquetitles.tolist()

In [57]:
#def createEmployeeTitlesdf(title, df):
#    newdf = df[df['Job Title']==title]
#    newdf.reset_index(drop=True, inplace=True)
#    newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)    
#    return newdf.to_dict()

In [58]:
#DictList = {}

#for i in range(len(Titleslist)):
#    newdict = createEmployeeTitlesdf(Titleslist[i], EmployeeNameandID)
#    DictList[i] = newdict

In [59]:
#dflist = []

#for i in range(len(DictList)):
#    df = pd.DataFrame(DictList[i])
#    dflist.append(df)

In [60]:
#finalemployeedf = pd.concat(dflist).reset_index(drop=True)

In [61]:
#employeelisttomerge = finalemployeedf[['Name', 'PositionID']]

In [62]:
#employeelisttomerge.to_excel('listwithposition.xlsx')

In [63]:
#len(employeelisttomerge)

In [64]:
#df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [65]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)
df1['Client'].replace('', np.nan, inplace=True)
df1['Rollup Account'].replace('', np.nan, inplace=True)

In [66]:
df1['Initiative'].fillna("No Initiative", inplace=True)
df1['Project'].fillna("No Project", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("No GL Account", inplace=True)
df1['Memo'].fillna("No Memo", inplace=True)
df1['Transaction Number'].fillna("No Transaction Number", inplace=True)
df1['Vendor Name'].fillna("No Vendor Name", inplace=True)
df1['Vendor Rate'].fillna("No Vendor Rate", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("No Contract Expiration Date", inplace=True)
df1['Program'].fillna("No Program", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)
df1['Client'].fillna("No Client", inplace=True)
df1['Rollup Account'].replace("New Account", np.nan, inplace=True)

## Add Further Heather Calcs

In [67]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [68]:
df1['Inst or Reclass'].value_counts()

7           93689
8           16677
5            6316
6            2862
7Reclass     1002
8Reclass      243
5Reclass       77
9               2
Name: Inst or Reclass, dtype: Int64

In [69]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [70]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [71]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [72]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [73]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [74]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [75]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [76]:
df1["Rev or Exp"].value_counts()

Expense    111611
Revenue      9255
Name: Rev or Exp, dtype: int64

In [77]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [78]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [79]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [80]:
#df1 = df1.drop('Match ID Names',1)

## Output manipulated DF to Excel

In [81]:
#df1.to_excel('Output_Netsuite.xlsx')

# 3.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [82]:
Programs = list(df1['Program'].unique())

In [83]:
print(Programs)

['Development', 'Islands', 'Operations', 'India', 'China', 'Breakthrough Technology', 'Carbon-Free Mobility', 'Climate Aligned Industries', 'Global Climate Finance', 'Carbon-Free Electricity', 'Strategic Analysis and Engagement', 'Carbon-Free Buildings', 'Climate Intelligence', 'WattTime', 'Urban Transformation', 'Building Electrification', 'Industry', 'Africa', 'SE Asia', 'US', 'CFAN', 'Energy Transition Academy', 'Communications', 'Cross-Program Impact Acceleration', 'Canary Media Program', 'No Program']


In [84]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [85]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Operations = create_program('Operations')
China = create_program('China')
Breakthrough_Technology = create_program('Breakthrough Technology')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Global Climate Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Analysis and Engagement')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')
Energy_Transition_Academy = create_program('Energy Transition Academy')
Communications = create_program('Communications')
CFAN = create_program('CFAN')


Development DF is EMPTY
Communications DF is EMPTY


In [86]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia, Energy_Transition_Academy, CFAN], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)


## 4. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [87]:
import os

path = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Detail_Trans_Reports'

In [88]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = "Detail_Transactions_"+datem.strftime("%b_%Y")

In [89]:
print(timestamp+".xlsx")

Detail_Transactions_Apr_2021.xlsx


In [90]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

## 5.  Create Pivot Table, Add Subtotals, Export to Excel, and Combine All These Functions into One

In [91]:
def create_multi_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [92]:
def add_subtotal_multi(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [93]:
def to_excel(pivot, str):
    pivot.to_excel(path+str, freeze_panes = (1,9), engine = 'openpyxl', sheet_name = 'Restricted_Rev_Detail', float_format = "%.2f")

In [94]:


def edit_workbook(file):
    book = load_workbook(file)
    sheet = book.active #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
#    sheet.protection.sheet = True
    book.save(file)

## Testing for unmerging cells

In [95]:
# THIS CODE WILL UNMERGE ALL THE CELLS IN A SHEET.   
def unmerge_rows(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['Restricted_Rev_Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [96]:
def format_multi_pivot(Group, str):
    step1 = create_multi_pivot(Group)
    step2 = add_subtotal_multi(step1)
    to_excel(step2, str)
    edit_workbook(path+str)
    unmerge_rows(path+str)

## 6.  Create Restricted Revenue Files

In [97]:

%%time
format_multi_pivot(India, '\\India_'+timestamp+".xlsx")
format_multi_pivot(Operations, '\\Operations_'+timestamp+".xlsx")
format_multi_pivot(China, '\\China_'+timestamp+".xlsx")
format_multi_pivot(Breakthrough_Technology, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Mobility, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_multi_pivot(Climate_Aligned_Industries, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_multi_pivot(Strategic_Engagement_and_Analysis_Group, '\\Strategic_Analysis_and_Engagement_Group_'+timestamp+".xlsx")
format_multi_pivot(CarbonFreeBuildings_Rollup, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_multi_pivot(WattTime, '\\WattTime_'+timestamp+".xlsx")
format_multi_pivot(CarbonFree_Electricty, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_multi_pivot(Urban_Transformation, '\\Urban_Transformation_'+timestamp+".xlsx")
format_multi_pivot(Climate_Intelligence, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_multi_pivot(Industry, '\\Industry_'+timestamp+".xlsx")
format_multi_pivot(Developing_Economies_Rollup, '\\Developing_Economies_'+timestamp+".xlsx")
format_multi_pivot(US, '\\US_'+timestamp+".xlsx")  
format_multi_pivot(Global_Climate_Finance, '\\Global_Climate_Finance_'+timestamp+".xlsx")  
#format_multi_pivot(Communications, '\\Communications_'+timestamp+".xlsx")  

Wall time: 10min 13s


## 7.  Add ML and Consulting Fee Tab

In [98]:
def create_df_FeeforService(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='5')|(firstdf['Account Digit']=='6')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [99]:
DevelopmentFFS = create_df_FeeforService("Development")
IslandsFFS = create_df_FeeforService("Islands")
IndiaFFS = create_df_FeeforService("India")
OperationsFFS = create_df_FeeforService("Operations")
ChinaFFS = create_df_FeeforService("China")
Breakthrough_Technology_FFS = create_df_FeeforService("Breakthrough Technology")
CarbonFree_MobilityFFS = create_df_FeeforService("Carbon-Free Mobility")
ClimateAligned_IndustriesFFS = create_df_FeeforService("Climate Aligned Industries")
CommunicationsFFS = create_df_FeeforService("Communications")
GlobalClimate_FinanceFFS = create_df_FeeforService("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupFFS = create_df_FeeforService("Strategic Analysis and Engagement")
Carbon_Free_BuildingsFFS = create_df_FeeforService("Carbon-Free Buildings")
Watttime_FFS = create_df_FeeforService("WattTime")
Carbon_Free_ElectricityFFS = create_df_FeeforService("Carbon-Free Electricity")
UrbanTrans_FFS = create_df_FeeforService("Urban Transformation")
ClimateIntelFFS = create_df_FeeforService("Climate Intelligence")
IndustryFFS = create_df_FeeforService("Industry")
SEAsiaFFS = create_df_FeeforService("SE Asia")
AfricaFFS = create_df_FeeforService("Africa")
USFFS = create_df_FeeforService("US")
BuildingElectrificationFFS = create_df_FeeforService("Building Electrification")
Energy_Transition_AcademyFFS = create_df_FeeforService("Energy Transition Academy")
CFANFFS = create_df_FeeforService("CFAN")

Communications DF is EMPTY


In [100]:
Developing_Economies_RollupFFS = pd.concat([IslandsFFS, AfricaFFS, SEAsiaFFS, Energy_Transition_AcademyFFS, CFANFFS], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupFFS = pd.concat([Carbon_Free_BuildingsFFS, BuildingElectrificationFFS], ignore_index = True, sort=False)


In [101]:
def create_FFS_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Client', 'Transaction Number'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [102]:
def add_subtotal_FFS(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '',  'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '',  '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [103]:
def to_excel_FFS(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="ML & FFS Revenue")
    writer.save()
    writer.close()    

In [104]:
def edit_workbook_FFS(file):
    book = load_workbook(file)
    sheet = book["ML & FFS Revenue"] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Client"
    sheet['G1']= "Transaction Number" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(8, 21):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 8):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=7).value == "Subtotal":
            for j in range(8,21):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "H2"
    book.save(file)

In [105]:
def unmerge_rows_FFS(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=7).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['ML & FFS Revenue'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [106]:
def format_FFS_pivot(Group, str):
    step1 = create_FFS_pivot(Group)
    step2 = add_subtotal_FFS(step1)
    to_excel_FFS(step2, path+str)
    edit_workbook_FFS(path+str)
    unmerge_rows_FFS(path+str)

## 8.  Add ML and FFS Pivots to Second Tab

In [107]:

format_FFS_pivot(Developing_Economies_RollupFFS, '\\Developing_Economies_'+timestamp+".xlsx")
format_FFS_pivot(IndiaFFS, '\\India_'+timestamp+".xlsx")
format_FFS_pivot(OperationsFFS, '\\Operations_'+timestamp+".xlsx")
format_FFS_pivot(ChinaFFS, '\\China_'+timestamp+".xlsx")
format_FFS_pivot(Breakthrough_Technology_FFS, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_FFS_pivot(CarbonFree_MobilityFFS, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_FFS_pivot(ClimateAligned_IndustriesFFS, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_FFS_pivot(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic_Analysis_and_Engagement_Group_'+timestamp+".xlsx")
format_FFS_pivot(CarbonFreeBuildings_RollupFFS, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_FFS_pivot(Watttime_FFS, '\\WattTime_'+timestamp+".xlsx")
format_FFS_pivot(Carbon_Free_ElectricityFFS, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_FFS_pivot(UrbanTrans_FFS, '\\Urban_Transformation_'+timestamp+".xlsx")
format_FFS_pivot(ClimateIntelFFS, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_FFS_pivot(IndustryFFS, '\\Industry_'+timestamp+".xlsx")
format_FFS_pivot(USFFS, '\\US_'+timestamp+".xlsx")
format_FFS_pivot(GlobalClimate_FinanceFFS, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_FFS_pivot(DevelopmentFFS, '\\Development_'+timestamp+".xlsx")

# 9.  Total Expenses

In [108]:
def create_df_allExpenses(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='7')|(firstdf['Account Digit']=='8')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [109]:
DevelopmentExp = create_df_allExpenses("Development")
IslandsExp = create_df_allExpenses("Islands")
IndiaExp = create_df_allExpenses("India")
OperationsExp = create_df_allExpenses("Operations")
ChinaExp = create_df_allExpenses("China")
Breakthrough_Technology_Exp = create_df_allExpenses("Breakthrough Technology")
CarbonFree_MobilityExp = create_df_allExpenses("Carbon-Free Mobility")
ClimateAligned_IndustriesExp = create_df_allExpenses("Climate Aligned Industries")
CommunicationsExp = create_df_allExpenses("Communications")
GlobalClimate_FinanceExp = create_df_allExpenses("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupExp = create_df_allExpenses("Strategic Analysis and Engagement")
Carbon_Free_BuildingsExp = create_df_allExpenses("Carbon-Free Buildings")
Watttime_Exp = create_df_allExpenses("WattTime")
Carbon_Free_ElectricityExp = create_df_allExpenses("Carbon-Free Electricity")
UrbanTrans_Exp = create_df_allExpenses("Urban Transformation")
ClimateIntelExp = create_df_allExpenses("Climate Intelligence")
IndustryExp = create_df_allExpenses("Industry")
SEAsiaExp = create_df_allExpenses("SE Asia")
AfricaExp = create_df_allExpenses("Africa")
USExp = create_df_allExpenses("US")
BuildingElectrificationExp = create_df_allExpenses("Building Electrification")
Energy_Transition_AcademyExp = create_df_allExpenses("Energy Transition Academy")
CFANExp = create_df_allExpenses("CFAN")

In [110]:
Developing_Economies_RollupExp = pd.concat([IslandsExp, CFANExp, AfricaExp, SEAsiaExp, Energy_Transition_AcademyExp], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupExp = pd.concat([Carbon_Free_BuildingsExp, BuildingElectrificationExp], ignore_index = True, sort=False)


In [111]:
def create_pivot_Exp(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'GL Account', 'Memo', 'Transaction Number', 'Vendor Name', 'Vendor Rate', 'Vendor Contract Expiration Date'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [112]:
def add_subtotal_Exp(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '', '', '', '', '', '', '', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '', '', '', '', '', '', '', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [113]:
def to_excel_Exp(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="All Expense Detail")
    writer.save()
    writer.close()    

In [114]:
def edit_workbook_Exp(file):
    book = load_workbook(file)
    sheet = book['All Expense Detail'] #active means last opened sheet
    sheet['A1']= "Initiative"
    sheet['B1']= "Project"
    sheet['C1']= "Grant"
    sheet['D1']= "GL Account"
    sheet['E1']= "Memo"
    sheet['F1']= "Transaction Number"
    sheet['G1']= "Vendor Name"
    sheet['H1']= "Vendor Rate"
    sheet['I1']= "Vendor Contract Expiration Date" 
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet['F1'].font = Font(bold=True)
    sheet['G1'].font = Font(bold=True)
    sheet['H1'].font = Font(bold=True)
    sheet['I1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 15
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 20
    sheet.column_dimensions['D'].width = 20
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 12.5
    sheet.column_dimensions['K'].width = 12.5
    sheet.column_dimensions['L'].width = 12.5
    sheet.column_dimensions['M'].width = 12.5
    sheet.column_dimensions['N'].width = 12.5
    sheet.column_dimensions['O'].width = 12.5
    sheet.column_dimensions['P'].width = 12.5
    sheet.column_dimensions['Q'].width = 12.5
    sheet.column_dimensions['R'].width = 12.5
    sheet.column_dimensions['S'].width = 12.5
    sheet.column_dimensions['T'].width = 12.5   
    sheet.column_dimensions['U'].width = 12.5   
    sheet.column_dimensions['V'].width = 12.5   
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 1
    for col in range(10, 23):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0.00_);[Red]("$"#,##0.00)'
    for col in range(1, 10):
        for row in range(1, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Subtotal":
            for j in range(10,23):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))       
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 55
    sheet.freeze_panes = "J2"
    book.save(file)

In [115]:
def unmerge_rows_EXP(file):
    for col in range(1,5):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=9).value == "Subtotal":
                testsheet.cell(row=i, column=col).value = None
        book.save(file) 
        book = load_workbook(file)
        testsheet = book['All Expense Detail'] #active means last opened sheet
        for i in range(1, testsheet.max_row + 1):
            if testsheet.cell(row=i, column=1).value == "Grand":
                testsheet.cell(row=i, column=2).value = None
                testsheet.cell(row=i, column=3).value = None
                testsheet.cell(row=i, column=4).value = None
        book.save(file) 

In [116]:
def format_Exp_pivot(Group, str):
    step1 = create_pivot_Exp(Group)
    step2 = add_subtotal_Exp(step1)
    to_excel_Exp(step2, path+str)
    edit_workbook_Exp(path+str)
    unmerge_rows_EXP(path+str)

In [117]:

format_Exp_pivot(Developing_Economies_RollupExp, '\\Developing_Economies_'+timestamp+".xlsx")
format_Exp_pivot(IndiaExp, '\\India_'+timestamp+".xlsx")
format_Exp_pivot(OperationsExp, '\\Operations_'+timestamp+".xlsx")
format_Exp_pivot(ChinaExp, '\\China_'+timestamp+".xlsx")
format_Exp_pivot(Breakthrough_Technology_Exp, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_Exp_pivot(CarbonFree_MobilityExp, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_Exp_pivot(ClimateAligned_IndustriesExp, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_Exp_pivot(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic_Analysis_and_Engagement_Group_'+timestamp+".xlsx")
format_Exp_pivot(CarbonFreeBuildings_RollupExp, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_Exp_pivot(Watttime_Exp, '\\WattTime_'+timestamp+".xlsx")
format_Exp_pivot(Carbon_Free_ElectricityExp, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_Exp_pivot(UrbanTrans_Exp, '\\Urban_Transformation_'+timestamp+".xlsx")
format_Exp_pivot(ClimateIntelExp, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_Exp_pivot(IndustryExp, '\\Industry_'+timestamp+".xlsx")
format_Exp_pivot(USExp, '\\US_'+timestamp+".xlsx")
format_Exp_pivot(GlobalClimate_FinanceExp, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_Exp_pivot(DevelopmentExp, '\\Development_'+timestamp+".xlsx")
format_Exp_pivot(CommunicationsExp, '\\Communications_'+timestamp+".xlsx")

## 10. Add Mavenlink Detail

In [118]:
def create_Mavenlinkprogram(program):
    firstdf = dfMavenlink[dfMavenlink['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [119]:
DevelopmentML = create_Mavenlinkprogram("Development")
IslandsML = create_Mavenlinkprogram("Islands")
IndiaML = create_Mavenlinkprogram("India")
OperationsML = create_Mavenlinkprogram("Operations")
ChinaML = create_Mavenlinkprogram("China")
Breakthrough_Technology_ML = create_Mavenlinkprogram("Breakthrough Technology")
CarbonFree_MobilityML = create_Mavenlinkprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesML = create_Mavenlinkprogram("Climate Aligned Industries")
CommunicationsML = create_Mavenlinkprogram("Communications")
GlobalClimate_FinanceML = create_Mavenlinkprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupML = create_Mavenlinkprogram("Strategic Analysis and Engagement")
Carbon_Free_BuildingsML = create_Mavenlinkprogram("Carbon-Free Buildings")
Carbon_Free_ElectricityML = create_Mavenlinkprogram("Carbon-Free Electricity")
UrbanTrans_ML = create_Mavenlinkprogram("Urban Transformation")
ClimateIntelML = create_Mavenlinkprogram("Climate Intelligence")
IndustryML = create_Mavenlinkprogram("Industry")
SEAsiaML = create_Mavenlinkprogram("SE Asia")
AfricaML = create_Mavenlinkprogram("Africa")
USML = create_Mavenlinkprogram("US")
BuildingElectrificationML = create_Mavenlinkprogram("Building Electrification")
Energy_Transition_AcademyML = create_Mavenlinkprogram("Energy Transition Academy")
CommunicationsML = create_Mavenlinkprogram("Communications")
CFANML = create_Mavenlinkprogram("CFAN")

In [120]:
Developing_Economies_RollupML = pd.concat([IslandsML, AfricaML, SEAsiaML, Energy_Transition_AcademyML, CFANML], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupML = pd.concat([Carbon_Free_BuildingsML, BuildingElectrificationML], ignore_index = True, sort=False)


In [121]:
def create_ML_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Employee','Case/Task/Event','Grant','Billable'], values = 'Percent of Total Monthly Hours', columns = 'Month', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [122]:
def to_excel_ML(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Mavenlink Hours")
    writer.save()
    writer.close()   

In [123]:
def edit_workbook_ML(file):
    book = load_workbook(file)
    sheet = book["Mavenlink Hours"] #active means last opened sheet
    sheet['B1']= "Employee"
    sheet['A1']= "Initiative"
    sheet['C1']= "Case/Task/Event"    
    sheet['D1']= "Grant"     
    sheet['E1']= "Billable"    
    sheet['A1'].font = Font(bold=True)
    sheet['B1'].font = Font(bold=True)
    sheet['C1'].font = Font(bold=True)
    sheet['D1'].font = Font(bold=True)
    sheet['E1'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['R'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(6, 19):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'
    for col in range(1, 6):
        for row in range(1, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = greyFill 
    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 60
    sheet.freeze_panes = "F2"
    book.save(file)

In [124]:
def unmerge_rows_ML(file):
    for col in range(1,4):
        rowstounmerge = []
        book = load_workbook(file)
        testsheet = book["Mavenlink Hours"] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                rowstounmerge.append(i)
        rowstounmerge.append(testsheet.max_row+1)
        for i in range(1,len(rowstounmerge)-1):        
            try:
                testsheet.unmerge_cells(start_row=rowstounmerge[i], start_column=col, end_row = rowstounmerge[i+1]-1, end_column=col) 
            except:
                pass
        book.save(file) 
        book = load_workbook(file)
        testsheet = book["Mavenlink Hours"] #active means last opened sheet
        for i in range(1,testsheet.max_row + 1):
            if testsheet.cell(row=i, column=col).value:
                continue
            else:
                testsheet.cell(row=i, column=col).value = testsheet.cell(row=i-1, column=col).value
        book.save(file) 


In [125]:
def format_pivot_ML(Group, str):
    step1 = create_ML_pivot(Group)
    to_excel_ML(step1, path+str)
    edit_workbook_ML(path+str)
    unmerge_rows_ML(path+str)

In [126]:

format_pivot_ML(Developing_Economies_RollupML, '\\Developing_Economies_'+timestamp+".xlsx")
format_pivot_ML(IndiaML, '\\India_'+timestamp+".xlsx")
format_pivot_ML(OperationsML, '\\Operations_'+timestamp+".xlsx")
format_pivot_ML(ChinaML, '\\China_'+timestamp+".xlsx")
format_pivot_ML(Breakthrough_Technology_ML, '\\Breakthrough_Technology-'+timestamp+".xlsx")
format_pivot_ML(CarbonFree_MobilityML, '\\CarbonFree_Mobility_'+timestamp+".xlsx")
format_pivot_ML(ClimateAligned_IndustriesML, '\\Climate_Aligned_Industries_'+timestamp+".xlsx")
format_pivot_ML(Strategic_Engagement_and_Analysis_GroupML, '\\Strategic_Analysis_and_Engagement_Group_'+timestamp+".xlsx")
format_pivot_ML(CarbonFreeBuildings_RollupML, '\\CarbonFree_Buildings_'+timestamp+".xlsx")
format_pivot_ML(Carbon_Free_ElectricityML, '\\CarbonFree_Electricity_'+timestamp+".xlsx")
format_pivot_ML(UrbanTrans_ML, '\\Urban_Transformation_'+timestamp+".xlsx")
format_pivot_ML(ClimateIntelML, '\\Climate_Intelligence_'+timestamp+".xlsx")
format_pivot_ML(IndustryML, '\\Industry_'+timestamp+".xlsx")
format_pivot_ML(USML, '\\US_'+timestamp+".xlsx")
format_pivot_ML(GlobalClimate_FinanceML, '\\Global_Climate_Finance_'+timestamp+".xlsx")
format_pivot_ML(DevelopmentML, '\\Development_'+timestamp+".xlsx")
format_pivot_ML(CommunicationsML, '\\Communications_'+timestamp+".xlsx")